In [ ]:
import pygrib
import pandas as pd
import numpy as np
import os

# Eindhoven
eindhoven_lat = 51.4416
eindhoven_lon = 5.4697

# Folder grib files
grib_folder = r"C:\Users\20193362\Desktop\harm40_v1_p1_2024052706"

# List needed parameters, see code matrix KNMI
parameters = ['11', '33', '34', '117'] 

# Initialize list to hold the data
data_list = []

# Loop over each file 
for file_name in os.listdir(grib_folder):
    if file_name.endswith('_GB'):
        grib_file = os.path.join(grib_folder, file_name)
        grbs = pygrib.open(grib_file)
        
        # Retrieve the lat/lon grid
        first_message = grbs.message(1)
        lats, lons = first_message.latlons()
        
        # Find the closest grid point
        distance = np.sqrt((lats - eindhoven_lat)**2 + (lons - eindhoven_lon)**2)
        min_index = distance.argmin()
        nearest_point_lat = lats.flat[min_index]
        nearest_point_lon = lons.flat[min_index]
        
        # Initialize a dictionary to hold the data for this file
        data_dict = {
            'file_name': file_name,
            'latitude': nearest_point_lat,
            'longitude': nearest_point_lon
        }
        
        # Extract data for each parameter
        for param in parameters:
            try:
                grb_message = grbs.select(parameterName=param)[0] # First instance
                parameter_name = param
                eindhoven_value = grb_message.values.flat[min_index]
                data_dict[parameter_name] = eindhoven_value
            except (IndexError, ValueError):
                data_dict[parameter_name] = np.nan # When parameter is not found in grib file
        
        grbs.close()
        
        # Append dictionary to list
        data_list.append(data_dict)

# Convert list of dictionaries to DF
all_data = pd.DataFrame(data_list)

print(all_data)